### 2. Setup

In [9]:
%autoawait
import time
import asyncio
from termcolor import colored,cprint

from aries_basic_controller.aries_controller import AriesAgentController

WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://seller-agent:8021"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL)


IPython autoawait is `on`, and set to use `asyncio`


In [10]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

### 3. We can get the credential by the credential_id used when storing

In [11]:
credential_id = "My Device Credential"
credential = await agent_controller.credentials.get_by_id(credential_id)
print(credential)

{'referent': 'My Device Credential', 'attrs': {'Status': '1', 'ExpirationDate': '1704242006', 'Manufacturer': 'GOOD_COMPANY', 'Owner': 'GOOD_SELLER', 'ProductionDate': '1672706006', 'Type': 'Dentistry'}, 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:Device_schema3:0.0.1', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:40:default', 'rev_reg_id': 'PQRXDxdGqQGSZ8z69p4xZP:4:PQRXDxdGqQGSZ8z69p4xZP:3:CL:40:default:CL_ACCUM:19a59ddb-6263-4f08-a0d1-94df3355f179', 'cred_rev_id': '1'}


### 4. Setup Listener

In [12]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def proof_handler(payload):
    print("Handle present proof")
    role = payload["role"]
    pres_ex_id = payload["presentation_exchange_id"]
    state = payload["state"]
    print(f"Role {role}, Exchange {pres_ex_id} in state {state}")
          
proof_listener = {
    "topic": "present_proof",
    "handler": proof_handler
}
agent_controller.register_listeners([proof_listener], defaults=True)

Subscribing too: present_proof
Handle present proof
Role prover, Exchange aa11ac50-81f3-494a-af1d-550c62d075da in state request_received


### 5. Go to Manufacturer Notebook to send verify request

### 8. Get verify request

In [16]:
response = await agent_controller.proofs.get_records()
print(response)

print('\n')

state = response['results'][0]["state"]
presentation_exchange_id = response['results'][0]['presentation_exchange_id']
presentation_request = response['results'][0]['presentation_request']

print('Presentation Exchange ID\n')
print(response['results'][0]['presentation_exchange_id'])
print('Presentation Request Object\n')
print(response['results'][0]['presentation_request'])
print('Requested Attributes\n')
print(response['results'][0]['presentation_request']['requested_attributes'])
requested_attribs = response['results'][0]['presentation_request']['requested_attributes']
print('Requested Predicates\n')
print(response['results'][0]['presentation_request']['requested_predicates'])
requested_predicates = response['results'][0]['presentation_request']['requested_predicates']

{'results': [{'initiator': 'external', 'trace': False, 'state': 'presentation_acked', 'presentation_request': {'name': 'Proof of Personal Information', 'version': '1.0', 'requested_attributes': {'0_ProductionDate_uuid': {'name': 'ProductionDate', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_Manufacturer_uuid': {'name': 'Manufacturer', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_Owner_uuid': {'name': 'Owner', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_Type_uuid': {'name': 'Type', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}}, 'requested_predicates': {'0_ExpirationDate_GE_uuid': {'name': 'ExpirationDate', 'p_type': '>=', 'p_value': 1672706059, 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_Status_GE_uuid': {'name': 'Status', 'p_type': '>', 'p_value': 0, 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}}, 'non_revoked': {'to': 1672706059}, 'nonce': '603376102097717995219671'}, 'presentati

### 9. Select credential from wallet

In [17]:
if state == "request_received":
    print(
    "Received Request -> Query for credentials in the wallet that satisfy the proof request")

# include self-attested attributes (not included in credentials)
credentials_by_reft = {}
revealed = {}
self_attested = {}
predicates = {}

# select credentials to provide for the proof
credentials = await agent_controller.proofs.get_presentation_credentials(presentation_exchange_id)
print(credentials)

if credentials:
    for row in sorted(
        credentials,
        key=lambda c: dict(c["cred_info"]["attrs"]),
        reverse=True,
    ):
        for referent in row["presentation_referents"]:
            if referent not in credentials_by_reft:
                credentials_by_reft[referent] = row

for referent in presentation_request["requested_attributes"]:
    if referent in credentials_by_reft:
        revealed[referent] = {
            "cred_id": credentials_by_reft[referent]["cred_info"][
                "referent"
            ],
            "revealed": True,
        }
    else:
        self_attested[referent] = "South Africa"

for referent in presentation_request["requested_predicates"]:
    if referent in credentials_by_reft:
        predicates[referent] = {
            "cred_id": credentials_by_reft[referent]["cred_info"][
                "referent"
            ]
        }

print("\nGenerate the proof")
proof = {
    "requested_predicates": predicates,
    "requested_attributes": revealed,
    "self_attested_attributes": self_attested,
}
print(proof)
print("\nXXX")
print(predicates)
print(revealed)
print(self_attested)

[{'cred_info': {'referent': 'My Device Credential', 'attrs': {'Owner': 'GOOD_SELLER', 'Type': 'Dentistry', 'Manufacturer': 'GOOD_COMPANY', 'ExpirationDate': '1704242006', 'Status': '1', 'ProductionDate': '1672706006'}, 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:Device_schema3:0.0.1', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:40:default', 'rev_reg_id': 'PQRXDxdGqQGSZ8z69p4xZP:4:PQRXDxdGqQGSZ8z69p4xZP:3:CL:40:default:CL_ACCUM:19a59ddb-6263-4f08-a0d1-94df3355f179', 'cred_rev_id': '1'}, 'interval': {'from': None, 'to': 1672706059}, 'presentation_referents': ['0_Type_uuid', '0_Status_GE_uuid', '0_Owner_uuid', '0_ProductionDate_uuid', '0_ExpirationDate_GE_uuid', '0_Manufacturer_uuid']}]

Generate the proof
{'requested_predicates': {'0_ExpirationDate_GE_uuid': {'cred_id': 'My Device Credential'}, '0_Status_GE_uuid': {'cred_id': 'My Device Credential'}}, 'requested_attributes': {'0_ProductionDate_uuid': {'cred_id': 'My Device Credential', 'revealed': True}, '0_Manufacturer_uuid': {'cred_id': 'My

### 10. Send proof back

In [ ]:
response = await agent_controller.proofs.send_presentation(presentation_exchange_id, proof)
print(response)

### **Terminate**

In [8]:
response = await agent_controller.terminate()